In [1]:
import torch
import numpy as np
import sys
import time

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [4]:
class SimIonization(torch.nn.Module):

    def __init__(self):
        super(SimIonization, self).__init__()

        self._density = 1.38    # g/cm^3
        self._alpha   = 0.847
        self._beta    = 0.2061
        self._efield  = 0.500   # V/sm
        self._lifetime = 6000.  # ms
        self._energy_threshold = 0.06 # MeV threshold to ignore drift
        self._dedx_threshold   = 0.0001 # MeV/cm threshold to ignore ...
        self._vdrift  = 0.153812 # cm/us
        
    def forward(self,x):
        '''
        Input: x tensor w/ shape (N,3) where 2 = (E,x,de/dx)
        Return: (N,1) where 1 = (Q,x,de/dx)
        '''
        # apply recombination
        x[:,0] *= torch.log(self._alpha + self._beta * x[:,2]) / (self._beta * x[:,2])
        # apply lifetime
        x[:,1] *= torch.exp( -1. * x[:,1] / self._vdrift / self._lifetime)

        return x

## Converting numpy arrays into Tensors

In [43]:
%%time
npts = int(1e8)
de = np.random.random(npts) * 0.9 + 0.1
dedx = np.random.random(npts) * 0.5 + 2.0
x  = np.random.random(npts) * 200 + 30.
ar = torch.Tensor(np.column_stack([de,x,dedx]))
print('Prepared a tensor',ar.size())

Prepared a tensor torch.Size([100000000, 3])
CPU times: user 4.78 s, sys: 1.24 s, total: 6.02 s
Wall time: 4.73 s


In [44]:
%%time
sim = SimIonization()
result = sim(ar)
print('Done',result)

Done tensor([[2.3566e-01, 1.7078e+02, 2.2894e+00],
        [7.1291e-02, 1.6644e+02, 2.3275e+00],
        [3.2906e-01, 1.1768e+02, 2.1446e+00],
        ...,
        [2.9505e-01, 1.3582e+02, 2.3626e+00],
        [5.1798e-01, 1.6613e+02, 2.2790e+00],
        [4.5030e-01, 1.1797e+02, 2.1126e+00]])
CPU times: user 2.2 s, sys: 859 ms, total: 3.06 s
Wall time: 1.61 s


## Using FloatTensors

In [15]:
%%time
npts = int(1e8)
de = torch.FloatTensor(npts).uniform_(0.1, 1)
dedx = torch.FloatTensor(npts).uniform_(1.5, 2.5)
x = torch.FloatTensor(npts).uniform_(30, 230)
ar = torch.stack([de, x, dedx], dim=1)
print('Prepared a tensor',ar.size())

Prepared a tensor torch.Size([100000000, 3])
CPU times: user 6.94 s, sys: 410 ms, total: 7.35 s
Wall time: 7.22 s


In [45]:
%%time 
sim = SimIonization()
result = sim(ar)
print('Done',result)

Done tensor([[1.3822e-01, 1.4193e+02, 2.2894e+00],
        [4.2013e-02, 1.3897e+02, 2.3275e+00],
        [1.8900e-01, 1.0359e+02, 2.1446e+00],
        ...,
        [1.7459e-01, 1.1724e+02, 2.3626e+00],
        [3.0341e-01, 1.3876e+02, 2.2790e+00],
        [2.5725e-01, 1.0381e+02, 2.1126e+00]])
CPU times: user 2.2 s, sys: 858 ms, total: 3.06 s
Wall time: 1.64 s


## Using FloatTensors on the GPU

In [39]:
%%time
npts = int(1e8)
de = torch.cuda.FloatTensor(npts).uniform_(0.1, 1)
dedx = torch.cuda.FloatTensor(npts).uniform_(1.5, 2.5)
x = torch.cuda.FloatTensor(npts).uniform_(30, 230)
ar = torch.stack([de, x, dedx], dim=1)
print('Prepared a tensor',ar.size())

Prepared a tensor torch.Size([100000000, 3])
CPU times: user 3.62 ms, sys: 3.34 ms, total: 6.96 ms
Wall time: 6.56 ms


In [41]:
%%time 
sim = SimIonization()
result = sim(ar)
print('Done',result)

Done tensor([[6.6116e-02, 1.3512e+02, 1.9368e+00],
        [3.9215e-02, 1.3814e+02, 1.8728e+00],
        [1.1456e-01, 1.4458e+02, 1.8342e+00],
        ...,
        [1.4807e-01, 5.6999e+01, 1.5469e+00],
        [7.0041e-02, 1.0304e+02, 1.5254e+00],
        [6.8445e-02, 9.2175e+01, 2.0959e+00]], device='cuda:0')
CPU times: user 27.9 ms, sys: 0 ns, total: 27.9 ms
Wall time: 27.1 ms


In [124]:
N, D_in, D_out = 1000, 1, 2

torch.nn.Linear(D_in, D_out)
x = torch.randn(N, D_in)

In [91]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(x[0])

print(sum(torch.abs(torch.floor(x/0.01))).int())
      
y = torch.zeros([sum(torch.abs(torch.floor(x/0.01))).int(), 1])
y.size()

tensor([-0.3389])
tensor([78510], dtype=torch.int32)


torch.Size([78510, 1])

In [121]:

class IonizationElectrons(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(OneLayerNet, self).__init__()

    def _numberOfClusters(step):
        return step*100./6.
        
    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        y_pred = self.linear(x).clamp(min=0,max=0)
        print("x",x)
        return y_pred

In [122]:
model = Ionia

In [123]:
print(x.size())
model(x).size()

torch.Size([1000, 1])
x tensor([[-3.3891e-01],
        [-1.4496e-01],
        [ 6.3848e-01],
        [ 5.9394e-01],
        [ 7.0710e-01],
        [ 8.4145e-01],
        [ 8.9842e-02],
        [ 7.3735e-01],
        [ 6.8615e-01],
        [ 8.3734e-01],
        [-2.5146e-01],
        [-4.1489e-02],
        [-1.3305e+00],
        [-1.3016e+00],
        [-1.5273e-01],
        [ 4.8555e-01],
        [ 7.9082e-01],
        [ 1.2275e+00],
        [ 1.0547e+00],
        [ 8.3393e-01],
        [-8.3203e-01],
        [ 1.1247e+00],
        [-1.7249e-01],
        [ 1.5231e+00],
        [ 6.0056e-01],
        [ 3.6476e-01],
        [-1.4430e+00],
        [-3.1154e-01],
        [-2.2617e+00],
        [ 3.7639e-01],
        [-3.7605e-01],
        [-7.8812e-01],
        [ 3.4509e-01],
        [-1.8162e+00],
        [ 2.4868e-01],
        [ 4.6476e-01],
        [-6.3422e-01],
        [ 4.5347e-01],
        [ 4.0835e-01],
        [ 1.6084e+00],
        [-1.8880e+00],
        [-2.9842e-01],
        [-

torch.Size([1000, 10])